# Bài toán

> **Thu thập dữ liệu báo Việt Nam (VnExpress - Demo)**

Mục tiêu:

- Hiểu được quy trình thu thập dữ liệu từ các trang báo Việt Nam
- Thu thập dữ liệu (bài báo) từ các trang báo Việt Nam để làm dữ liệu cho các bước xử lý sau

Đầu ra:

-  Tập file JSON chứa các bài bài báo có các trường dữ liệu:

    - `url`: link dẫn đến bài báo
    - `title`: tiêu đề bài báo
    - `description`: tóm tắt bài báo
    - `content`: nội dung bài báo
    - `metadata`: trường dữ liệu bổ sung

        - `cat`: thể loại bài báo
        - `subcat`: thể loại con của bài báo
        - `published_date`: thời gian xuất bản
        - `author`: người viết
    
- Ví dụ về một bài báo:

    ```
    {
        "url": "https://vnexpress.net/chinh-phu-ban-hanh-nghi-dinh-moi-ve-gia-dat-4763835.html",
        "title": "Chính phủ ban hành nghị định mới về giá đất",
        "description": "Chính phủ hôm nay ban hành Nghị định 71, trong đó quy ...",
        "content": "Nghị định này có hiệu lực khi Luật Đất đai 2024 được thi hành ...",
        "metadata": {
            "cat": "Bất động sản",
            "subcat": "Chính sách",
            "published_date": 1719575647,
            "author": "Anh Tú"
        }
    },
    ```

# Các bước tiến hành

## Chuẩn bị các thư viện cần thiết

In [15]:
# Suitable for Google Colab, for local please follow the external instructions and ignore this line
# and follows https://docs.google.com/document/d/14jK9d6KHJYX0b-gFAVqAghUxT7OLAM0nP2IovL7_Rjs/edit?usp=sharing
!apt install -qq chromium-chromedriver

'apt' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
# Install selenium
!pip install -qq selenium

# Tạo thư mục để chứa data
!mkdir -p data

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file data already exists.
Error occurred while processing: data.


In [1]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# other imports
import os
import json

In [2]:
# selenium setups
## https://www.tutorialspoint.com/selenium/selenium_webdriver_chrome_webdriver_options.htm

chrome_options = webdriver.ChromeOptions()

# chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [3]:
MAGAZINE_NAME = "vnexpress"
HOME_PAGE = "https://vnexpress.net/"

## Thu thập dữ liệu

> **Các bước thu thập bài báo**


1. News categories: Thu thập tất cả các thể loại báo của website
2. News urls: Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó
3. News articles: Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước

### Thu thập thể loại bài báo của website: Crawling categories

> **Các bước thu thập**

1. Vào trang chủ của báo
2. Thu thập các thể loại ở mục menu


Vào trang chủ

In [4]:
driver = webdriver.Chrome(options=chrome_options)
# Vào trang web chính, mặc định phải chờ toàn bộ trang webload mới xong
driver.get(HOME_PAGE)

Chọn menu buttons

In [8]:
# Chọn menu buttons

## Chọn element theo class_name == "all-menu"
all_menu = driver.find_element(by=By.CLASS_NAME, value="all-menu")

## Click vào menu
all_menu.click() #

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=140.0.7339.128); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementnotinteractableexception
Stacktrace:
	GetHandleVerifier [0x0x7ff6103cfc95+79861]
	GetHandleVerifier [0x0x7ff6103cfcf0+79952]
	(No symbol) [0x0x7ff61014c8fc]
	(No symbol) [0x0x7ff6101a59c3]
	(No symbol) [0x0x7ff610197338]
	(No symbol) [0x0x7ff6101ccada]
	(No symbol) [0x0x7ff610196bc6]
	(No symbol) [0x0x7ff6101cccf0]
	(No symbol) [0x0x7ff6101f4f8b]
	(No symbol) [0x0x7ff6101cc8b3]
	(No symbol) [0x0x7ff610195272]
	(No symbol) [0x0x7ff610196043]
	GetHandleVerifier [0x0x7ff61068b9cd+2946349]
	GetHandleVerifier [0x0x7ff610685c4a+2922410]
	GetHandleVerifier [0x0x7ff6106a59d7+3052855]
	GetHandleVerifier [0x0x7ff6103eaa7e+189918]
	GetHandleVerifier [0x0x7ff6103f2a1f+222591]
	GetHandleVerifier [0x0x7ff6103d8ab4+116244]
	GetHandleVerifier [0x0x7ff6103d8c69+116681]
	Ge

Thu thập hết thể loại:

* Cách chọn elements ở web trong selenium: https://selenium-python.readthedocs.io/locating-elements.html

In [9]:
# Lấy hết tất cả thể loại từ đây
cats = []

# Chọn row-menu bằng class_name
row_menu = driver.find_element(by=By.CLASS_NAME, value="row-menu")

# Ta thấy mỗi thể loại ở trong element là cat-menu element --> ta lấy hết elements bằng dùng find_elements; find_element -> chọn element đầu tiên match
cat_menus = row_menu.find_elements(by=By.CLASS_NAME, value="cat-menu")

for cat_menu in cat_menus: # Loop through cat menus to main cat and corresponding links
    # Lấy element đầu tiên
    cat = cat_menu.find_element(by=By.TAG_NAME, value="a").get_attribute("title").strip()
    href = cat_menu.find_element(by=By.TAG_NAME, value="a").get_attribute("href").strip()
    # Lưu vào cats list bao gồm tên và url
    cats.append({"cat_name": cat, "url": href})


In [11]:
cats, len(cats)

([{'cat_name': 'VnE-GO', 'url': 'https://vnexpress.net/vne-go'},
  {'cat_name': 'Thời sự', 'url': 'https://vnexpress.net/thoi-su'},
  {'cat_name': 'Thế giới', 'url': 'https://vnexpress.net/the-gioi'},
  {'cat_name': 'Kinh doanh', 'url': 'https://vnexpress.net/kinh-doanh'},
  {'cat_name': 'Khoa học công nghệ',
   'url': 'https://vnexpress.net/khoa-hoc-cong-nghe'},
  {'cat_name': 'Giải trí', 'url': 'https://vnexpress.net/giai-tri'},
  {'cat_name': 'Pháp luật', 'url': 'https://vnexpress.net/phap-luat'},
  {'cat_name': 'Góc nhìn', 'url': 'https://vnexpress.net/goc-nhin'},
  {'cat_name': 'Bất động sản', 'url': 'https://vnexpress.net/bat-dong-san'},
  {'cat_name': 'Sức khỏe', 'url': 'https://vnexpress.net/suc-khoe'},
  {'cat_name': 'Thể thao', 'url': 'https://vnexpress.net/the-thao'},
  {'cat_name': 'Ý kiến', 'url': 'https://vnexpress.net/y-kien'},
  {'cat_name': 'Tâm sự', 'url': 'https://vnexpress.net/tam-su'},
  {'cat_name': 'Giáo dục', 'url': 'https://vnexpress.net/giao-duc'},
  {'cat_nam

In [12]:
# Đóng driver sau khi dùng xong
driver.close()

### Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó: News urls


> **Cách thu thập**

Từ những thể loại và đường link tương ứng, ta lần lượt vào từng thể loại và lấy đường dẫn của các bài báo trong mục đó của trang web đó.



#### Cài đặt thông số

In [13]:
# Đặt số lượng đường dẫn cần lấy trong mỗi thể loại báo
NUM_ARTICLES_PER_CAT = 30 # có thể tăng lên

# Đường dẫn lưu data của vnexpress
DATA_URL_FILE = "data/vnexpress_url.json"

# Một số thể loại không quan tâm
EXCLUDING_CATEGORIES = ["Video", "Podcasts", "Góc nhìn", "Tâm sự", "Thư giãn", "Ý kiến"]

# Bổ sung cài đặt chromedriver
## Ta đặt load stategy ở đây là normal: https://www.selenium.dev/documentation/webdriver/drivers/options/
chrome_options.page_load_strategy = "normal"

#### Chạy thử nghiệm

In [35]:
driver = webdriver.Chrome(options=chrome_options)

In [36]:
# Vào thử một url
driver.get(cats[0]['url'])

In [37]:
# Get all item-news class item
title_news = driver.find_elements(by=By.CLASS_NAME, value="title-news")

In [39]:
# Lấy đường dẫn
url_new = title_news[0].find_element(by=By.TAG_NAME, value="a").get_attribute("href")
url_new

'https://vnexpress.net/thu-tuong-lap-quy-hoc-bong-quoc-gia-tang-chi-cho-giao-duc-4939531.html'

In [40]:
# Chỉ lấy links bắt đầu giống HOME_PAGE
url_new.startswith(HOME_PAGE)

True

In [41]:
# Tìm next page
driver.find_element(by=By.CLASS_NAME, value="next-page").get_attribute("href")

'https://vnexpress.net/thoi-su-p2'

In [ ]:
driver.close()

#### Chạy thật


In [43]:
driver = webdriver.Chrome(options=chrome_options)

In [50]:
# Global variables for filtering deduplicating urls
crawled_urls = set()

def crawl_each_category_url(driver, category_url):
    """
    Functions cho lấy urls cho từng category sau khi thử nghiệm
    """
    all_urls = set()
    url = category_url

    # Limit the number of NUM_ARTICLES_PER_CAT

    while len(all_urls) < NUM_ARTICLES_PER_CAT:
        driver.get(url)
        title_news = driver.find_elements(by=By.CLASS_NAME, value="title-news")
        for title in title_news:
            try:
                url_new = title.find_element(by=By.TAG_NAME, value="a").get_attribute("href")
                if url_new.startswith(HOME_PAGE) and url_new not in crawled_urls: #avoid ads, different sites news
                    all_urls.add(url_new)
                    crawled_urls.add(url_new) # avoid dedup url

            # To see if there is bug
            except StaleElementReferenceException:
                continue
            except NoSuchElementException:
                print(f"NoSuchElementException at {url}")
                continue

        try:
            url = driver.find_element(by=By.CLASS_NAME, value="next-page").get_attribute("href")
        except NoSuchElementException:
            break

    return all_urls

In [51]:
saved_cats = {}

# Thu thập cho từng thể loại
for cat in cats:
    cat_name = cat["cat_name"]
    url = cat["url"]
    if cat_name not in EXCLUDING_CATEGORIES:
        print(f"You are at {cat}.")
        urls = crawl_each_category_url(driver, url)
        saved_cats[cat_name] = list(urls)

with open(DATA_URL_FILE, "w", encoding='utf-8') as fOut:
    json.dump(saved_cats, fOut, ensure_ascii=False, indent=4)

driver.close()

You are at {'cat_name': 'Thời sự', 'url': 'https://vnexpress.net/thoi-su'}.
You are at {'cat_name': 'Thế giới', 'url': 'https://vnexpress.net/the-gioi'}.
You are at {'cat_name': 'Kinh doanh', 'url': 'https://vnexpress.net/kinh-doanh'}.
You are at {'cat_name': 'Khoa học công nghệ', 'url': 'https://vnexpress.net/khoa-hoc-cong-nghe'}.
You are at {'cat_name': 'Giải trí', 'url': 'https://vnexpress.net/giai-tri'}.
You are at {'cat_name': 'Pháp luật', 'url': 'https://vnexpress.net/phap-luat'}.
You are at {'cat_name': 'Bất động sản', 'url': 'https://vnexpress.net/bat-dong-san'}.
You are at {'cat_name': 'Sức khỏe', 'url': 'https://vnexpress.net/suc-khoe'}.
You are at {'cat_name': 'Thể thao', 'url': 'https://vnexpress.net/the-thao'}.
You are at {'cat_name': 'Giáo dục', 'url': 'https://vnexpress.net/giao-duc'}.
You are at {'cat_name': 'Đời sống', 'url': 'https://vnexpress.net/doi-song'}.
You are at {'cat_name': 'Xe', 'url': 'https://vnexpress.net/oto-xe-may'}.
You are at {'cat_name': 'Du lịch', '

In [53]:
len(crawled_urls)

652

### Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước: News articles


> **Cách thu thập**

Từ đường dẫn ở trong phần trước, ta lần lượt vào từng đường link đó và thu thập thông tin về bài báo.

#### Cài đặt thông số

In [57]:
# Filepath cho cái trước
FILE_URL_PATH = "data/vnexpress_url.json"

# Đặt limit số articles lấy từ mỗi thể loại
MAX_ARTICLES_PER_CAT = 2 # Nếu set = None thì là tất cả các urls ở file trước

# Data output, mỗi thể loại là 1 file json chứa articles
DATA_FOLDER_OUTPUT = "data/vnexpress"
#
# !mkdir -p $DATA_FOLDER_OUTPUT
from pathlib import Path
Path(DATA_FOLDER_OUTPUT).mkdir(parents=True, exist_ok=True)

# Để loading strategy về eager load nhanh, không quan tâm ảnh
chrome_options.page_load_strategy = "eager"

In [60]:
# Đọc file url
with open(FILE_URL_PATH, "r", encoding='utf-8') as fIn:
    url_data = json.load(fIn)

len(url_data)

13

#### Chạy thử nghiệm

In [86]:
driver = webdriver.Chrome(options=chrome_options)

# Một số url để thử nghiệm
SAMPLE_ARTICLE_URLS = [
    "https://vnexpress.net/lao-tuyen-bo-quoc-tang-tuong-niem-tong-bi-thu-nguyen-phu-trong-4772895.html",
    "https://vnexpress.net/ty-le-hoc-sinh-do-tot-nghiep-cao-nhat-10-nam-4772931.html",
    "https://vnexpress.net/tra-luong-cho-nguoi-gia-4772632.html",
    "https://vnexpress.net/cuoc-song-do-thi-mien-nam-hon-100-nam-truoc-4772382.html",
    "https://vnexpress.net/piastri-lan-dau-thang-chang-f1-4772734.html", # table
    "https://vnexpress.net/10-ung-vien-gianh-qua-bong-vang-2024-4772973.html", # slideshow,
    "https://vnexpress.net/toi-kho-chiu-khi-nhieu-nguoi-dung-xe-khi-con-ba-giay-den-xanh-4772885.html", # author,
    "https://vnexpress.net/khong-vo-gao-truoc-khi-nau-co-an-toan-4772624.html",
    "https://vnexpress.net/do-ban-thay-con-cong-khac-biet-trong-10-giay-4770515.html",
]

# chọn url
SAMPLE_ARTICLE_URL = SAMPLE_ARTICLE_URLS[0]

In [87]:
driver.get(SAMPLE_ARTICLE_URL)

In [88]:
# Tìm kiếm title
driver.find_element(by=By.CSS_SELECTOR, value="h1.title-detail").text

'Lào tuyên bố quốc tang tưởng niệm Tổng Bí thư Nguyễn Phú Trọng'

In [89]:
# Tìm kiếm description
driver.find_element(by=By.CLASS_NAME, value="description").text

'Chính phủ Lào thông báo sẽ tổ chức quốc tang ngày 25-26/7 để tưởng niệm Tổng Bí thư Nguyễn Phú Trọng.'

In [90]:
# Thu thập thể loại
lis_cat = driver.find_element(by=By.CSS_SELECTOR, value="ul.breadcrumb").find_elements(by=By.TAG_NAME, value="li")
main_cat = lis_cat[0].text if len(lis_cat) > 0 else None
sub_cat = lis_cat[1].text if len(lis_cat) > 1 else None
main_cat, sub_cat

('Thế giới', None)

In [91]:
# Thu thập ngày
publish_date = driver.find_element(by=By.CSS_SELECTOR, value='[itemprop="datePublished"]').get_attribute("content").strip()
publish_date

'2024-07-22T14:50:18+07:00'

In [92]:
# Tìm kiếm contents
article = driver.find_element(by=By.CSS_SELECTOR, value="article.fck_detail")
children = article.find_elements(by=By.XPATH, value="./*")

In [93]:
# Thu thập contents và author
contents = []
author = "Unknown"
is_slide_show = False

for idx, child in enumerate(children):
    text = child.text.strip()
    # right align
    if child.tag_name == "p" and ("right" in child.get_attribute("align") or "right" in child.get_attribute("style")) and idx >= len(children) - 3: # last three, align right --> author
        author = text
    elif child.tag_name == "p" and child.get_attribute("class") == "Normal": # paragraph
        # If center
        if len(text):
            if ("center" in child.get_attribute("align") or "center" in child.get_attribute("style")):
                contents.append(f"[{text}]")
            else:
                contents.append(text)
    elif child.tag_name == "figure" :
        ## If length > 100  --> not a caption, it's next description
        if len(text):
            if len(text) <= 100:
                contents.append(f"[{text}]")
            else:
                contents.append(text)
    elif child.tag_name == "div" and "item_slide_show" in child.get_attribute("class"):
        is_slide_show = True # slideshow
        if len(text):
            if len(text) <= 100:
                contents.append(f"[{text}]")
            else:
                contents.append(text)

    elif child.tag_name == "table": # Do nothing rightnow
        pass

if is_slide_show:
    author = text

if author == "Unknown":
    try:
        author = driver.find_element(by=By.XPATH, value="//*[contains(@class, 'author')]").text
    except:
        pass

In [94]:
contents

['Trong cuộc họp báo tại Văn phòng Thủ tướng Lào hôm nay, Bộ trưởng, Chủ nhiệm Văn phòng Thủ tướng Lào Bouakhong Nammavong thông báo Đảng và chính phủ Lào nhất trí để tang cấp quốc gia, bao gồm cả đại sứ quán Lào và các cơ quan đại diện ngoại giao ở nước ngoài, trong ngày 25-26/7, treo cờ rủ và dừng các hoạt động vui chơi ở mọi hình thức.',
 'Tổng Bí thư, Chủ tịch nước Lào Thongloun Sisoulith sẽ dẫn đầu đoàn lãnh đạo cấp cao tham dự lễ viếng và lễ tang Tổng Bí thư Nguyễn Phú Trọng tại Hà Nội. Chính phủ Lào cũng sẽ phối hợp chuẩn bị cho các đoàn đến viếng, ghi sổ tang và đặt vòng hoa tại trụ sở Đại sứ quán Việt Nam ở thủ đô Vientiane.',
 '[Tổng Bí thư Nguyễn Phú Trọng. Ảnh: VGP]',
 'Bộ Ngoại giao Lào được yêu cầu tổ chức Hội nghị Bộ trưởng Ngoại giao ASEAN lần thứ 57 và các hội nghị liên quan tại Lào ngày 21-27/7 theo cách thức phù hợp nhất và thông báo cho tất cả các phái đoàn tham gia hội nghị biết, đặc biệt là việc tổ chức tiệc và lễ hội.',
 'Các lãnh đạo Lào ngày 19/7 đã gửi điện ch

In [95]:
author

'Vũ Anh (Theo TTXVN)'

In [96]:
driver.close()

#### Chạy thật


In [102]:
def get_content_metadata(driver, article_url):

    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)

    # Thu thập title
    title = driver.find_element(by=By.CSS_SELECTOR, value="h1.title-detail").text.strip()

    # Thu thập description
    description = driver.find_element(by=By.CLASS_NAME, value="description").text.strip()

    # Thu thập thể loại
    lis_cat = driver.find_element(by=By.CSS_SELECTOR, value="ul.breadcrumb").find_elements(by=By.TAG_NAME, value="li")
    main_cat = lis_cat[0].text if len(lis_cat) > 0 else None
    sub_cat = lis_cat[1].text if len(lis_cat) > 1 else None

    # Thu thập published date
    publish_date = driver.find_element(by=By.CSS_SELECTOR, value='[itemprop="datePublished"]').get_attribute("content").strip()

    # Thu thập content bài báo
    # Locate phần viết content
    article = driver.find_element(by=By.CSS_SELECTOR, value="article.fck_detail")
    # Lấy hết các đầu mục con của bài báo
    children = article.find_elements(by=By.XPATH, value="./*")

    contents = []
    author = "Unknown"

    # Check có phải dạng slide show hay không
    is_slide_show = False
    for idx, child in enumerate(children):
        text = child.text.strip()
        # Nếu mà element right align --> có thể là tác giả
        if child.tag_name == "p" and ("right" in child.get_attribute("align") or "right" in child.get_attribute("style")) and idx >= len(children) - 3: # last three, align right --> author
            author = text
        elif child.tag_name == "p" and child.get_attribute("class") == "Normal": # paragraph
            # If center
            if len(text):
                if ("center" in child.get_attribute("align") or "center" in child.get_attribute("style")):
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Chỉ lấy caption của figure
        elif child.tag_name == "figure" :
            ## If length > 100  --> not a caption, it's next description
            if len(text):
                if len(text) <= 100: # nếu mà len <= 100 --> add thêm [] xung quanh
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Nếu mà là slide show thì nó giống figure
        elif child.tag_name == "div" and "item_slide_show" in child.get_attribute("class"):
            is_slide_show = True # slideshow
            if len(text):
                if len(text) <= 100:
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)

        # Bỏ qua table bây giờ
        elif child.tag_name == "table": # Do nothing rightnow
            pass

    if is_slide_show:
        author = text

    # Nếu mà vẫn chưa thấy author thì tìm bằng tag
    if author == "Unknown":
        try:
            author = driver.find_element(by=By.XPATH, value="//*[contains(@class, 'author')]").text
        except:
            pass

    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents), # join các đoạn bằng \n
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat,
            "published_date": publish_date,
            "author": author
        }
    }


In [140]:
driver = webdriver.Chrome(options=chrome_options)

for cat, urls in url_data.items():

    print(f"Thu thập dữ liệu thể loại {cat} ..")
    count_crawled = 0
    cat_data = []
    for url in urls:
        try:
            cat_data.append(get_content_metadata(driver, url))
            count_crawled += 1
            if MAX_ARTICLES_PER_CAT and count_crawled >= MAX_ARTICLES_PER_CAT:
                break

        except (StaleElementReferenceException, NoSuchElementException) as e:
            print(f"Bug at url: {url}, with ElementException")
            driver.refresh()
            # Chúng ta tạm bỏ lỗi bây giờ
            continue

    name_file_cat = cat.lower().replace(" ", "-") + ".json"

    with open(os.path.join(DATA_FOLDER_OUTPUT, name_file_cat), "w", encoding='utf-8') as fOut:
        json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()

Thu thập dữ liệu thể loại Thời sự ..
Thu thập dữ liệu thể loại Thế giới ..
Thu thập dữ liệu thể loại Kinh doanh ..
Thu thập dữ liệu thể loại Khoa học công nghệ ..
Thu thập dữ liệu thể loại Giải trí ..
Thu thập dữ liệu thể loại Pháp luật ..
Thu thập dữ liệu thể loại Bất động sản ..
Bug at url: https://vnexpress.net/bat-dong-san/du-an/detail/vinhomes-wonder-park-dan-phuong-547, with ElementException
Thu thập dữ liệu thể loại Sức khỏe ..
Thu thập dữ liệu thể loại Thể thao ..
Bug at url: https://vnexpress.net/haaland-qua-mat-cantona-rooney-o-derby-manchester-4939067.html, with ElementException
Thu thập dữ liệu thể loại Giáo dục ..
Thu thập dữ liệu thể loại Đời sống ..
Thu thập dữ liệu thể loại Xe ..
Bug at url: https://vnexpress.net/oto-xe-may/v-car/doanh-so/10-oto-ban-nhieu-nhat-thi-truong-viet-nam-thang-8-4939203.html, with ElementException
Thu thập dữ liệu thể loại Du lịch ..
Bug at url: https://vnexpress.net/du-lich-viet-tu-so-0-den-top-3-khu-vuc-4932801.html, with ElementException


In [132]:
for key in url_data.keys():
    print(f'{key} - {len(url_data[key])}')

Thời sự - 55
Thế giới - 53
Kinh doanh - 60
Khoa học công nghệ - 55
Giải trí - 48
Pháp luật - 47
Bất động sản - 44
Sức khỏe - 53
Thể thao - 45
Giáo dục - 59
Đời sống - 42
Xe - 31
Du lịch - 60


In [138]:
# Xem 1 sample
cat_data[0]

{'url': 'https://vnexpress.net/singapore-lam-moi-trai-nghiem-du-lich-mice-bang-du-thuyen-5-sao-4935202.html',
 'title': 'Singapore làm mới trải nghiệm du lịch MICE bằng du thuyền 5 sao',
 'description': 'Du thuyền tích hợp phòng hội nghị với không gian team building, thể thao mạo hiểm, giúp làm mới các hoạt động MICE (hội họp kết hợp nghỉ dưỡng) tại Singapore.',
 'content': '',
 'metadata': {'cat': 'Du lịch',
  'subcat': 'Điểm đến',
  'published_date': '2025-09-11T10:00:00+07:00',
  'author': 'Unknown'}}

## Lưu dữ liệu

Nếu bạn chạy ở máy cá nhân thì không cần, nhưng nếu mà chạy ở Colab thì nên lưu dữ liệu vào trong Google Drive


In [ ]:
# For Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set to your folder
FOLDER_SAVED_GOOGLE_COLAB = "/content/drive/MyDrive/crawl-news/"

# Copy
!cp -r data $FOLDER_SAVED_GOOGLE_COLAB